In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import h_key
from api_keys import f_key
from api_keys import n_key


# Output File (CSV)
output_data_file = "output_data/hotel.csv"

# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

### API Calls

In [2]:
# Create project dataframe (cities)
city_df = pd.read_csv("./Resources/Airport_Output.csv")
city_df.dropna(inplace = True) 
city_df.head()


,Unnamed: 0,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating,Airport Code
0,0,New York City,New York,40.712775,-74.005973,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.2,EWR
1,1,Los Angeles,California,34.052234,-118.243685,Los Angeles International Airport,"1 World Way, Los Angeles",3.5,LAX
2,2,Chicago,Illinois,41.878114,-87.629798,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6,ORD
3,3,Houston,Texas,29.760427,-95.369803,William P. Hobby Airport,"7800 Airport Blvd, Houston",4.0,HOU
4,4,Philadelphia,Pennsylvania,39.952584,-75.165222,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.3,PHL


In [3]:

airport_code=[]
cheapestTicketPrice=[]

#outbound trip call
#for index, row in city_df.iterrows():
    #code = row['Airport Code']
    #originplace='BWI'
    #destinationplace=code
    #departure_date="2021-03-15"
    #base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/"
    #tags=originplace + "-sky/" + destinationplace + "-sky/" + departure_date
    #url=base_url + tags
    #print(url)
    #querystring = {"inboundpartialdate":"2021-03-19"}

    #headers = {
        #'x-rapidapi-key': "e5ee88badfmsh65ec7588e95e6fcp181b1cjsn38212393a000",
        #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
        #}
    #try:
        #response = requests.request("GET", url, headers=headers, params=querystring)
        #response=response.json()
        #airport_code.append(code)
        #cheapestTicketPrice.append(response['Quotes'][0]['MinPrice'])
    #except:
        #cheapestTicketPrice.append('nan')


In [4]:
#city_df['Cheapest_Outbound_Airfare']=cheapestTicketPrice

In [5]:
#airport_code=[]
#cheapestTicketPrice=[]
#return trip
#for index, row in city_df.iterrows():
    #code = row['Airport Code']
    #destinationplace='IAD'
    #originplace=code
    #departure_date="2021-03-19"
    #base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/"
    #tags=originplace + "-sky/" + destinationplace + "-sky/" + departure_date
    #url=base_url + tags
    #print(url)
    #querystring = {"inboundpartialdate":"2021-03-19"}

    #headers = {
        #'x-rapidapi-key': "e5ee88badfmsh65ec7588e95e6fcp181b1cjsn38212393a000",
        #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
        #}
    #try:
        #response = requests.request("GET", url, headers=headers, params=querystring)
        #response=response.json()
        #airport_code.append(code)
        #cheapestTicketPrice.append(response['Quotes'][0]['MinPrice'])
    #except:
        #cheapestTicketPrice.append('nan')

In [6]:
#city_df['Cheapest_Return_Airfare']=cheapestTicketPrice
#city_df



In [13]:
#Kayla API LOOP
counter = 0
cities = []
results=[]

    

#For loop
for city,item in city_df.iterrows():
    counter=counter+1  
    #print(item)
    try:
        url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"
        #print(item["Lat"],item["Lng"]) 
        querystring = {"lat":item["Lat"],"lon":item["Lng"],"checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

        headers = {
        'x-rapidapi-key': h_key,
        'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        response = response.json()
    
#Reading the length of each ref
        nh = len(response['data']['body']['searchResults']['results'])

        for i in range(nh):

            Hotel = response['data']['body']['searchResults']['results'][i]['name']
            Rating = response['data']['body']['searchResults']['results'][i]['starRating']
            City = response['data']['body']['searchResults']['results'][i]['address']['locality']
            State = response['data']['body']['searchResults']['results'][i]['address']['region']
            Country =response['data']['body']['searchResults']['results'][i]['address']['countryCode']
            Stay_Rate = response['data']['body']['searchResults']['results'][i]['ratePlan']['price']['exactCurrent']
            
            results.append([Hotel, Rating,City, State, Country,Stay_Rate])
            
            time.sleep(2)
            
    except:
        
        print("Hotel Data Not Found")
   
               
print(results)


Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data No

In [14]:
hotel_df = pd.DataFrame(results,columns=["Hotel","Rating","City", "State", "Country","Price"])

hotel_df.to_csv('HOTEL_API.csv', index=False)

hotel_df

,Hotel,Rating,City,State,Country,Price


In [9]:
#nichole api
locations = city_df['City'].values.tolist()

In [ ]:
# TODO(uwana): Use your key here. Get it from yelp.com/developers
api_key= n_key
headers = {'Authorization': 'Bearer %s' % api_key}
# Using the business search API.
url='https://api.yelp.com/v3/businesses/search'
locations
# https://www.yelp.com/developers/documentation/v3/business_search
# All the attractions data
attractions = []
# Making a get request to the API for each location
for location in locations:
  # Use query paramaters that conform to the API: https://www.yelp.com/developers/documentation/v3/business_search
  params = {'term': 'attractions', 'location': location, 'limit': 1, 'sort_by': 'rating'}
  req = requests.get(url, params=params, headers=headers)
  data = json.loads(req.text)
  businesses = data['businesses']
  for business in businesses:
    business_info = {'location': location, 'name': business['name'], 'rating': business['rating'],
                    'review_count': business['review_count']}
    attractions.append(business_info)
attractions_json = json.dumps(attractions)
print(attractions_json)

In [ ]:
yelp_df = pd.DataFrame(attractions)
yelp_df.head(100)